<h2 align='center' style='color:blue'>Dropout Regularization In Deep Neural Network</h2>

This is a dataset that describes sonar chirp returns bouncing off different services. The 60 input variables are the strength of the returns at different angles. It is a **binary classification problem** that requires a model to differentiate rocks from metal cylinders.

Dataset information: https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)
Download it from here: https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("/kaggle/input/sonar-data-rocks/sonar_dataset.csv", header=None)
df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
24,0.0293,0.0644,0.0390,0.0173,0.0476,0.0816,0.0993,0.0315,0.0736,0.0860,...,0.0035,0.0052,0.0083,0.0078,0.0075,0.0105,0.0160,0.0095,0.0011,R
78,0.0231,0.0351,0.0030,0.0304,0.0339,0.0860,0.1738,0.1351,0.1063,0.0347,...,0.0106,0.0097,0.0022,0.0052,0.0072,0.0056,0.0038,0.0043,0.0030,R
110,0.0210,0.0121,0.0203,0.1036,0.1675,0.0418,0.0723,0.0828,0.0494,0.0686,...,0.0117,0.0101,0.0061,0.0031,0.0099,0.0080,0.0107,0.0161,0.0133,M
121,0.0162,0.0041,0.0239,0.0441,0.0630,0.0921,0.1368,0.1078,0.1552,0.1779,...,0.0173,0.0135,0.0114,0.0062,0.0157,0.0088,0.0036,0.0053,0.0030,M
62,0.0086,0.0215,0.0242,0.0445,0.0667,0.0771,0.0499,0.0906,0.1229,0.1185,...,0.0072,0.0054,0.0022,0.0016,0.0029,0.0058,0.0050,0.0024,0.0030,R


In [4]:
df.shape

(208, 61)

In [5]:
# check for nan values
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [6]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [7]:
df[60].value_counts() # label is not skewed

M    111
R     97
Name: 60, dtype: int64

In [8]:
X = df.drop(60, axis=1)
y = df[60]
y.head()

0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object

In [9]:
y = pd.get_dummies(y, drop_first=True)
y.sample(5) # R --> 1 and M --> 0

,R
154,0
16,1
94,1
18,1
49,1


In [10]:
y.value_counts()

R
0    111
1     97
dtype: int64

In [11]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [13]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
67,0.0368,0.0403,0.0317,0.0293,0.0820,0.1342,0.1161,0.0663,0.0155,0.0506,...,0.0058,0.0091,0.0160,0.0160,0.0081,0.0070,0.0135,0.0067,0.0078,0.0068
14,0.0124,0.0433,0.0604,0.0449,0.0597,0.0355,0.0531,0.0343,0.1052,0.2120,...,0.0078,0.0083,0.0057,0.0174,0.0188,0.0054,0.0114,0.0196,0.0147,0.0062
164,0.0163,0.0198,0.0202,0.0386,0.0752,0.1444,0.1487,0.1484,0.2442,0.2822,...,0.0027,0.0077,0.0026,0.0031,0.0083,0.0020,0.0084,0.0108,0.0083,0.0033
179,0.0394,0.0420,0.0446,0.0551,0.0597,0.1416,0.0956,0.0802,0.1618,0.2558,...,0.0118,0.0146,0.0040,0.0114,0.0032,0.0062,0.0101,0.0068,0.0053,0.0087
19,0.0126,0.0149,0.0641,0.1732,0.2565,0.2559,0.2947,0.4110,0.4983,0.5920,...,0.0153,0.0092,0.0035,0.0098,0.0121,0.0006,0.0181,0.0094,0.0116,0.0063


# Using Deep Learning Model

## Model without Dropout Layer

In [14]:
import tensorflow as tf
from tensorflow import keras

In [15]:
model = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 2ms/step - loss: 0.6905 - accuracy: 0.5641
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6682 - accuracy: 0.5897
Epoch 3/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6507 - accuracy: 0.6282
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6318 - accuracy: 0.7436
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6048 - accuracy: 0.7500
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5843 - accuracy: 0.7051
Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5514 - accuracy: 0.8013
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5161 - accuracy: 0.7821
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4715 - accuracy: 0.8205
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4488 - accuracy: 0.8077
Epoch 11/

In [16]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 5ms/step - loss: 0.8863 - accuracy: 0.7308


[0.8862795233726501, 0.7307692170143127]

Training Accuracy >>> Test Accuracy

In [17]:
y_pred = model.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 3ms/step
[6.8763484e-07 8.2528037e-01 9.5892042e-01 4.5987152e-05 9.9999988e-01
 9.9985385e-01 6.6098654e-01 9.9999994e-01 5.0099326e-05 1.0000000e+00]
[0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]


In [18]:
y_test[:10]

,R
186,0
155,0
165,0
200,0
58,1
34,1
151,0
18,1
202,0
62,1


In [19]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.81      0.76        27
           1       0.76      0.64      0.70        25

    accuracy                           0.73        52
   macro avg       0.74      0.73      0.73        52
weighted avg       0.73      0.73      0.73        52



### Model with Dropout Layer

In [20]:
modeld = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

modeld.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modeld.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 2ms/step - loss: 0.7315 - accuracy: 0.4231
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7115 - accuracy: 0.5192
Epoch 3/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6951 - accuracy: 0.5577
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7021 - accuracy: 0.5385
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7084 - accuracy: 0.5000
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7167 - accuracy: 0.4744
Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6886 - accuracy: 0.5321
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6794 - accuracy: 0.5769
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6785 - accuracy: 0.5897
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7053 - accuracy: 0.4744
Epoch 11/

In [21]:
modeld.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.4775 - accuracy: 0.7692


[0.4775312840938568, 0.7692307829856873]

Training Accuracy is still good but Test Accuracy Improved

In [22]:
y_pred = modeld.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 2ms/step
[1.7469584e-04 9.0040147e-01 8.7308830e-01 1.3933248e-02 9.9992418e-01
 9.0992349e-01 1.7435567e-01 9.9996173e-01 1.7162763e-02 9.9996495e-01]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [23]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.89      0.80        27
           1       0.84      0.64      0.73        25

    accuracy                           0.77        52
   macro avg       0.78      0.76      0.76        52
weighted avg       0.78      0.77      0.77        52



**You can see that by using dropout layer test accuracy increased from 0.77 to 0.81**